# Spatial$^{*}$ Microsimulation Urban Metabolism Model (SMUM)

## Case Study: Brussels

<div class="image123">
    <div class="imgContainer">
        <img src="./logos/UNEnvironment.png" alt="UNEP logo" style="width:200px">
    </div>
    <div class="imgContainer">
        <img src="./logos/GI-REC.png" alt="GI_REC logo" style="width:200px">
    </div>
</div>

# 1.d Micro-level Water demand model

[UN Environment](http://www.unep.org/)

In [1]:
import datetime; print(datetime.datetime.now())

2017-12-26 17:17:53.598205


**Notebook abstract**

A simple micro-level water demand model.

A simple micro-level water demand model. Similar to the [electricity demand model](Ac_MCMC_electricity.ipynb), the water demand model uses micro level consumption demand data for the construction of a table model. The table model describes simple rules for the construction of the proxy micro level sample data.  

## Prior water demand model

In [2]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from urbanmetabolism._scripts.micro import compute_categories, change_index

/usr/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Dataset

The data used for the construction of the household water demand model is
downscale from municipal-level data-sets from the Wallonie region.

This data to construct a water demand model a function of:
1. Building construction type (classified as house or apartment)
2. Building construction year
3. Head of household gender
4. Household size
5. Household income level

In [3]:
water_data = pd.read_csv('data/downscaled_data_be.csv', index_col=0)
water_data.loc[
    water_data.ConstructionType != 'appartements',
    'ConstructionType'] = 'House'

formula = "Water ~ Age + ConstructionType + ConstructionYear + HHSize + Income"

In [4]:
water_data.head()

,Age,ConstructionType,ConstructionYear,Energy,Gender,HHSize,Income,Waste,Water,w
0,70,House,1900,49,femme,2.61,28150.0,147.0,79,71.45
1,22,House,1977,49,femme,2.61,28150.0,147.0,79,71.45
2,28,House,1972,49,femme,2.61,28150.0,147.0,79,71.45
3,7,House,1900,49,femme,2.61,28150.0,147.0,79,71.45
4,33,House,1962,49,homme,2.61,28150.0,147.0,79,71.45


## Regression model

A weighted multiple regression model is defined and run with help
of the ‘statsmodels’ python library.

The model regression coefficients and their standard error are used
as the water demand model.

In [5]:
model_water = sm.WLS.from_formula(formula, water_data, weights=water_data.w)
model_results_water = model_water.fit()

In [6]:
model_results_water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  Water   R-squared:                       0.360
Model:                            WLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     2519.
Date:                Tue, 26 Dec 2017   Prob (F-statistic):               0.00
Time:                        17:17:54   Log-Likelihood:                -75359.
No. Observations:               22412   AIC:                         1.507e+05
Df Residuals:                   22406   BIC:                         1.508e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -5.2824      2.273     -2.324      0.020      -9.738      -0.826
ConstructionType[T.appartements]     1.1517      0.112     10.307      0.000       0.933       1.371
Age                                  0.0005      0.002      0.305      0.760      -0.003       0.004
ConstructionYear                     0.0157      0.001     13.527      0.000       0.013       0.018
HHSize                              10.3606      0.273     37.966      0.000       9.826      10.895
Income                               0.0010   1.27e-05     75.207      0.000       0.001       0.001
==============================================================================
Omnibus:                     2964.935   Durbin-Watson:                   0.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7638.192
Skew:                           0.748   Prob(JB):                         0.00
Kurtosis:                       5.438   Cond. No.                     1.28e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.28e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
params_water = change_index(model_results_water.params)
bse_water = change_index(model_results_water.bse)
water = pd.concat([params_water, bse_water], axis=1)
water.columns = ['co_mu', 'co_sd']

In [8]:
water.index = [
    'Intercept', 'ConstructionType', 'Age',
    'ConstructionYear', 'HHSize', 'Income',
]

In [9]:
water.loc[:, 'p'] = np.nan
water.loc[:,'mu'] = np.nan
water.loc[:,'sd'] = np.nan

water.loc['ConstructionType', 'dis'] = 'Bernoulli'
water.loc['Age', 'dis'] = 'Normal'
water.loc['ConstructionYear', 'dis'] = 'Poisson'
water.loc['HHSize', 'dis'] = 'Poisson'
water.loc['Income', 'dis'] = 'Gamma'

water.loc[:, 'ub'] = np.inf
water.loc[:, 'lb'] = 0
water.loc['Intercept', 'lb'] = -np.inf
water.loc['Age', 'ub'] = 85
water.loc['Age', 'lb'] = 20
water.loc['HHSize', 'ub'] = 8
water.loc['HHSize', 'lb'] = 1
water.loc['ConstructionYear', 'ub'] = 2035
water.loc['ConstructionYear', 'lb'] = 1800

water.loc['Intercept', 'p'] = water.loc['Intercept', 'co_mu']
water.loc['Intercept', 'dis'] = 'Deterministic'
water.loc['Intercept', 'co_mu'] = np.nan
water.loc['Intercept', 'co_sd'] = np.nan

In [10]:
water.index = ['w_'+i for i in water.index]

In [11]:
water.to_csv('data/table_water.csv')

In [12]:
water

,co_mu,co_sd,p,mu,sd,dis,ub,lb
w_Intercept,NaN,NaN,-5.282409,NaN,NaN,Deterministic,inf,-inf
w_ConstructionType,1.151750,0.111739,NaN,NaN,NaN,Bernoulli,inf,0.000000
w_Age,0.000516,0.001691,NaN,NaN,NaN,Normal,85.000000,20.000000
w_ConstructionYear,0.015676,0.001159,NaN,NaN,NaN,Poisson,2035.000000,1800.000000
w_HHSize,10.360614,0.272889,NaN,NaN,NaN,Poisson,8.000000,1.000000
w_Income,0.000957,0.000013,NaN,NaN,NaN,Gamma,inf,0.000000


<div class="image123">
    <div class="imgContainer">
        <img src="./logos/UNEnvironment.png" alt="UNEP logo" style="width:200px">
    </div>
    <div class="imgContainer">
        <img src="./logos/GI-REC.png" alt="GI_REC logo" style="width:200px">
    </div>
</div>

# 1.d Micro-level Water demand model

[UN Environment](http://www.unep.org/)

[Home](Welcome.ipynb)

[Next](Ba_GREGWT_Dynamic.ipynb) (2.a) Dynamic Sampling Model  and GREGWT